<a href="https://colab.research.google.com/github/KeerthikaNagarajan/Multilingual-Video-Translation-and-Subtitle-Generator/blob/main/Video_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multilingual Video Translation and Subtitle Generator

In [1]:
# Install required libraries
!pip install moviepy ffmpeg pydub googletrans==4.0.0-rc1 SpeechRecognition ipywidgets gTTS


In [2]:
import os
from moviepy.editor import VideoFileClip, AudioFileClip
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
import ipywidgets as widgets
from IPython.display import display, clear_output

# Upload your video file
from google.colab import files
uploaded = files.upload()

# Extract the video file name
video_filename = list(uploaded.keys())[0]


Saving fightclub.mp4 to fightclub.mp4


Language Selection

In [3]:
# Language dropdown widget
language_options = {
    "French": "fr", "German": "de", "Spanish": "es",
    "Hindi": "hi", "Japanese": "ja", "Arabic": "ar",
    "Chinese (Simplified)": "zh-cn"
}

language_selector = widgets.SelectMultiple(
    options=language_options.keys(),
    description="Select Languages:",
    style={"description_width": "initial"},
)
display(language_selector)


SelectMultiple(description='Select Languages:', options=('French', 'German', 'Spanish', 'Hindi', 'Japanese', '…

Processing

In [4]:
# Extract audio from video
video = VideoFileClip(video_filename)
audio_path = "extracted_audio.wav"
video.audio.write_audiofile(audio_path)

# Transcribe audio
recognizer = sr.Recognizer()
with sr.AudioFile(audio_path) as source:
    audio_data = recognizer.record(source)

try:
    original_text = recognizer.recognize_google(audio_data)
    print("Transcribed Text:", original_text)
except sr.UnknownValueError:
    print("Speech not recognized.")


MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.


Transcribed Text: gentlemen welcome to fight song the first rule of Fight Club is you do not talk about Fight Club second rule of Fight Club is you do not talk about Fight Club


Translation and Audio Generation



In [5]:
# Translate and generate audio
translator = Translator()
selected_langs = [language_options[lang] for lang in language_selector.value]

for lang_code in selected_langs:
    translation = translator.translate(original_text, dest=lang_code).text
    tts = gTTS(translation, lang=lang_code)
    audio_filename = f"translated_audio_{lang_code}.mp3"
    tts.save(audio_filename)
    print(f"Saved translated audio for {lang_code} as {audio_filename}")


Saved translated audio for es as translated_audio_es.mp3


Replace Audio in Video


In [6]:
# Merge each translated audio with the video
for lang_code in selected_langs:
    translated_audio_file = f"translated_audio_{lang_code}.mp3"
    translated_audio = AudioFileClip(translated_audio_file)

    final_video = video.set_audio(translated_audio)
    output_video_file = f"translated_video_{lang_code}.mp4"
    final_video.write_videofile(output_video_file, codec='libx264', audio_codec='aac')

    print(f"Saved translated video for {lang_code} as {output_video_file}")


Moviepy - Building video translated_video_es.mp4.
MoviePy - Writing audio in translated_video_esTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video translated_video_es.mp4



t: 100%|█████████▉| 548/549 [00:54<00:00,  9.76it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file fightclub.mp4, 6220800 bytes wanted but 0 bytes read,at frame 548/549, at time 18.27/18.28 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready translated_video_es.mp4
Saved translated video for es as translated_video_es.mp4


Subtitles

In [7]:
!apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [8]:
# Function to generate an SRT file for a given language
def generate_srt_file(translation_text, filename, duration):
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("1\n")
        f.write("00:00:00,000 --> 00:00:{:02d},000\n".format(int(duration)))
        f.write(translation_text.strip() + "\n")

# Process each selected language
for lang_code in selected_langs:
    # Translate the transcribed text
    translation = translator.translate(original_text, dest=lang_code).text

    # Save subtitle file
    srt_filename = f"subtitles_{lang_code}.srt"
    generate_srt_file(translation, srt_filename, video.duration)

    # Burn subtitles into the translated video
    output_video_file_with_subs = f"translated_video_with_subs_{lang_code}.mp4"
    intermediate_video_file = f"translated_video_{lang_code}.mp4"

    # FFmpeg command to embed subtitles
    command = f"ffmpeg -i '{intermediate_video_file}' -vf subtitles='{srt_filename}' -c:a copy '{output_video_file_with_subs}'"
    os.system(command)

    print(f"Saved video with subtitles for {lang_code} as {output_video_file_with_subs}")


Saved video with subtitles for es as translated_video_with_subs_es.mp4
